<a href="https://colab.research.google.com/github/goodakai/GEE/blob/main/GEE_mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# python code for mapping leucogranite based on GEE
# author: Dakai Guo,Ziye Wang
# contact: Ziye Wang (Email: ziyewang@cug.edu.cn)

# mount google cloud drive and GEE API
from google.colab import drive
drive.mount('/content/drive')
import ee
ee.Authenticate()
ee.Initialize(project='my-project')# GEE Project Title

# input shape file
aoi = ee.FeatureCollection("Asset ID/Asset name1")# uploaded vector file of the Himalayan orogenic belt region
hl_aoi = ee.FeatureCollection("Asset ID/Asset name2")# uploaded vector file of the Cuonadong dome region
unhl_aoi = ee.FeatureCollection("Asset ID/Asset name3")# uploaded vector file of the Himalayan orogenic belt area excluding the area of Cuonadong dome

# Sentinel2 cloud mask function
def maskS2clouds(image):
  qa = image.select('QA60');# cloud mask band
  cloudBitMask = 1 << 10;# the 10th digit of QA60 represents the probability of cloud presence
  cirrusBitMask = 1 << 11;# the 11th digit of QA60 represents the probability of cirrus presence
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0));
  return image.updateMask(mask).divide(100);# for consistency with the ASTER value range divided by 100

# input ASTER and Sentinel2 image
asterimage =  (ee.Image('Asset ID/Asset name3')# uploaded TIF file of atmospherically corrected ASTER image
             .multiply(100)# for consistency with the Sentinel2 value range multiply by 100
             .clip(aoi)
             .select(['green','red','nir','swir1','swir2','swir3','swir4','swir5','swir6'],
                 ['A01','A02','A3N','A04','A05','A06','A07','A08','A09']));

sentinel2image =  (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')# calling Sentinel-2 image data from GEE cloud servers.
                   .filterDate('2020-01-01','2022-12-31')# filtering the date range in which the image was taken
                   .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10))# filtering images with less than 10% cloud parameter
                   .filterBounds(aoi)
                   .map(maskS2clouds)
                   .median()
                   .clip(aoi)
                   .reproject('EPSG:4326',null,100)# resampling of sentinel2 data to EPSG:4326 and 100 m resolution
                   .select(['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12']));

# select remote sensing bands
asterbands = asterimage.select('A3N','A04','A06','A07')
sentinel2bands = sentinel2image.select('B11')
alldata = asterbands.addBands(sentinel2bands)
bands = alldata.bandNames()

# production of train data
positive_po = ee.FeatureCollection.randomPoints(hl_aoi,4000)# 4000 randomly selected pixels in the leucogranite area of Cuonadng
negative_po = ee.FeatureCollection.randomPoints(unhl_aoi,4000)# # 4000 randomly selected pixels in unknown area

def add_1(feature):
    return feature.set('Lithology', 1)
positive_data = positive_po.map(add_1)# add label 1 to positive data

def add_0(feature):
    return feature.set('Lithology', 0)
negative_data = negative_po.map(add_0)# add label 0 to negative data

train_points = positive_data.merge(negative_data)
train_data = alldata.sampleRegions(**{
    'collection': train_points,
    'properties': ['Lithology'],# copy label values to the property list
    'scale': 100 # spatial resolution for extracting pixel values
})

# training the random forest model
withRandom = train_data.randomColumn('random')
split = 0.7 # set a threshold for splitting the dataset
trainingPartition = withRandom.filter(ee.Filter.lt('random', split))
testingPartition = withRandom.filter(ee.Filter.gte('random', split))

rf = ee.Classifier.smileRandomForest(15).setOutputMode('PROBABILITY').train(**{
    'features': trainingPartition,
    'classProperty': 'Lithology',
    'inputProperties': bands
})# construct a rf model with 15 trees and set it to PROBABILITY output mode

# test model performance
validated = testingPartition.classify(rf)

def probToClass(feature):
    prob = ee.Number(feature.get('classification'))
    label = prob.gte(0.5).toInt()# the results of the testingPartition were split according to a threshold of 0.5
    return feature.set('predicted', label)

validatedClass = validated.select('Lithology','classification').map(probToClass)
confusionMatrix = validatedClass.errorMatrix('Lithology','predicted')# calculate the confusionMatrix
print('confusionMatrix',confusionMatrix.getInfo())
print('overall accuracy', confusionMatrix.accuracy().getInfo())
print('kappa accuracy', confusionMatrix.kappa().getInfo())

# output the mapping results
img_classfication = alldata.classify(rf)
task = ee.batch.Export.image.toDrive(image=img_classfication,
                   region=aoi.geometry(),
                   description='description',# exporting task descriptions
                   folder='folder',# export to Google Cloud Drive folder
                   fileNamePrefix='fileName',# name of the exported file
                   scale=100,# resolution of the exported file
                   crs='EPSG:4326')# coordinate of the exported file

task.start()